## COMP5625M Assessment 2 - Image Caption Generation [100 marks]

<div class="logos"><img src="./Comp5625M_logo.jpg" width="220px" align="right"></div>

The maximum marks for each part are shown in the section headers. The overall assessment carries a total of 100 marks.

This assessment is weighted 25% of the final grade for the module.

### Motivation 

Through this assessment, you will:

> 1. Understand the principles of text pre-processing and vocabulary building.
> 2. Gain experience working with an image-to-text model.
> 3. Use and compare two text similarity metrics for evaluating an image-to-text model, and understand evaluation challenges.


### Setup and resources 

Having a GPU will speed up the image feature extraction process. If you want to use a GPU, please refer to the module website for recommended working environments with GPUs.

Please implement the coursework using PyTorch and Python-based libraries, and refer to the notebooks and exercises provided.

This assessment will use a subset of the [COCO "Common Objects in Context" dataset](https://cocodataset.org/) for image caption generation. COCO contains 330K images of 80 object categories, and at least five textual reference captions per image. Our subset consists of nearly 5070 of these images, each with five or more different descriptions of the salient entities and activities, and we will refer to it as COCO_5070.

To download the data:

> 1. **Images and annotations**: download the zipped file provided in the link here as [``COMP5625M_data_assessment_2.zip``](https://leeds365-my.sharepoint.com/:u:/g/personal/scssali_leeds_ac_uk/EWWzE-_AIrlOkvOKxH4rjIgBF_eUx8KDJMPKM2eHwCE0dg?e=DdX62H). 

``Info only:`` To understand more about the COCO dataset, you can look at the [download page](https://cocodataset.org/#download). We have already provided you with the "2017 Train/Val annotations (241MB)", but our image subset consists of fewer images than the original COCO dataset. **So, no need to download anything from here!** 

> 2. **Image metadata**: as our set is a subset of the full COCO dataset, we have created a CSV file containing relevant metadata for our particular subset of images. You can also download it from Drive, "coco_subset_meta.csv", at the same link as 1.


### Submission

Please submit the following:

> 1. Your completed Jupyter notebook file, in .ipynb format. **Do not change the file name.**
> 2. The .html version of your notebook; File > Download as > HTML (.html). Check that all cells have been run and all outputs (including all graphs you would like to be marked) are displayed in the .html for marking.

**Final note:**

> **Please include everything you would like to be marked in this notebook, including figures. Under each section, put the relevant code containing your solution. You may re-use functions you defined previously, but any new code must be in the appropriate section.** Feel free to add as many code cells as you need under each section.

Your student username (for example, ```sc15jb```):

Your full name:

### Imports

Feel free to add to this section as needed.

In [1]:
# check the python version
import sys
sys.executable

'E:\\ananconda\\envs\\torch\\python.exe'

In [2]:
pip freeze >requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import os
import numpy as np

Detect which device (CPU/GPU) to use.

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


The basic principle of our image-to-text model is as pictured in the diagram below, where an Encoder network encodes the input image as a feature vector by providing the outputs of the last convolutional layer of a pre-trained CNN (we use [ResNet50](https://arxiv.org/abs/1512.03385)). This pretrained network has been trained on the complete ImageNet dataset and is thus able to recognise common objects. 

**(Hint)** You can alternatively use the COCO trained pretrained weights from [PyTorch](https://pytorch.org/vision/stable/models.html). One way to do this is use the "FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1" but use e.g., "resnet_model = model.backbone.body". Alternatively, you can use the checkpoint from your previous coursework where you finetuned to COCO dataset. 

These features are then fed into a Decoder network along with the reference captions. As the image feature dimensions are large and sparse, the Decoder network includes a linear layer which downsizes them, followed by *a batch normalisation layer* to speed up training. Those resulting features, as well as the reference text captions, are then passed into a recurrent network (we will use **RNN** in this assessment). 

The reference captions used to compute loss are represented as numerical vectors via an **embedding layer** whose weights are learned during training.

<!-- ![Encoder Decoder](comp5625M_figure.jpg) --> 


<div>
<center><img src="comp5625M_figure_imageCaption.jpg" width="1000"/></center>
</div>


The Encoder-Decoder network could be coupled and trained end-to-end, without saving features to disk; however, this requires iterating through the entire image training set during training. We can make the **training more efficient by decoupling the networks**. Thus, we will:

> First extract the feature representations of the images from the Encoder

> Save these features (Part 1) such that during the training of the Decoder (Part 3), we only need to iterate over the image feature data and the reference captions.

**Hint**
Try commenting out the feature extraction part once you have saved the embeddings. This way if you have to re-run the entire codes for some reason then you can only load these features. 


### Overview

> 1. Extracting image features 
> 2. Text preparation of training and validation data 
> 3. Training the decoder
> 4. Generating predictions on test data
> 5. Caption evaluation via BLEU score
> 6. Caption evaluation via Cosine similarity
> 7. Comparing BLEU and Cosine similarity


## 1 Extracting image features [11 marks]

> 1.1 Design a encoder layer with pretrained ResNet50 (4 marks)

> 1.2 Image feature extraction step (7 marks)

#### 1.1 Design a encoder layer with pretrained ResNet50 (4 marks)

> Read through the template EncoderCNN class below and complete the class.

> You are expected to use ResNet50 pretrained on imageNet provided in the Pytorch library (torchvision.models)


In [5]:
class EncoderCNN(nn.Module):
    def __init__(self):
        """Load the pretrained ResNet-50 and replace top fc layer."""
        super(EncoderCNN, self).__init__()
        # Your code here!

        # TO COMPLETE
        # keep all layers of the pretrained net except the last layers of fully-connected ones (you are permitted to take other layers too but this can affect your accuracy!)

        # Load the pretrained ResNet-50 model
        resnet = models.resnet50(pretrained=True)
        
        # Remove the last (classification) layer
        modules = list(resnet.children())[:-1]
        
        # Create the modified ResNet-50 model
        self.resnet = nn.Sequential(*modules)
        
    def forward(self, images):
        """Extract feature vectors from input images."""

        # TO COMPLETE
        # remember no gradients are needed

        # No gradients needed during forward pass
        with torch.no_grad():
            features = self.resnet(images)

        # Flatten the output
        features = features.view(features.size(0), -1)
        
        return features 


In [6]:
# instantiate encoder and put into evaluation mode.
# Your code here!

# Instantiate the EncoderCNN
cnnencoder = EncoderCNN().to(device)

# Set the encoder to evaluation mode
cnnencoder.eval()

E:\ananconda\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\ananconda\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EncoderCNN(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64

#### 1.2 Image feature extraction step (7 marks)

Pass the images through the ```Encoder``` model, saving the resulting features for each image. You may like to use a ```Dataset``` and ```DataLoader``` to load the data in batches for faster processing, or you may choose to simply read in one image at a time from disk without any loaders.

Note that as this is a forward pass only, no gradients are needed. You will need to be able to match each image ID (the image name without file extension) with its features later, so we suggest either saving a dictionary of image ID: image features, or keeping a separate ordered list of image IDs.

Use this ImageNet transform provided.

In [7]:
data_transform = transforms.Compose([ 
    transforms.ToTensor(),
    transforms.Resize(224), 
    transforms.CenterCrop(224), 
    transforms.Normalize((0.485, 0.456, 0.406),   # using ImageNet norms
                         (0.229, 0.224, 0.225))])

In [8]:
# Get unique images from the csv for extracting features - helper code
imageList = pd.read_csv("./data/coco_subset_meta.csv")
imageList['file_name']
len(imageList.id.unique())

imagesUnique = sorted(imageList['file_name'].unique())
print(len(imagesUnique))

df_unique_files =  pd.DataFrame.from_dict(imagesUnique)

df_unique_files.columns = ['file_name']
df_unique_files

5068


,file_name
0,000000000009.jpg
1,000000000025.jpg
2,000000000030.jpg
3,000000000034.jpg
4,000000000036.jpg
...,...
5063,000000581906.jpg
5064,000000581909.jpg
5065,000000581913.jpg
5066,000000581921.jpg


In [9]:
# Define a class COCOImagesDataset(Dataset) function that takes the 
# image file names and reads the image and apply transform to it
# ---> your code here! we have provided you a sketch 

# import package to load the image 
from PIL import Image

IMAGE_DIR = "./data/coco/images/"

class COCOImagesDataset(Dataset):
    def __init__(self, df, transform=None):

        self.df = df
        # --> your code here!
        # create the transform
        self.transform = transform

    def __getitem__(self, index):
        filename = self.df.iloc[index]['file_name']

        # --> your code here!
        # create the image-path
        image_path = os.path.join(IMAGE_DIR, filename)

        # load the image
        image = Image.open(image_path).convert('RGB')

        # do the transform on the image
        if self.transform:
            image = self.transform(image)

        return image, filename

    def __len__(self):
        return len(self.df)
    

In [10]:
# Use Dataloader to use the unique files using the class COCOImagesDataset
# make sure that shuffle is False as we are not aiming to retrain in this exercise
# Your code here-->
# import useful package
from torch.utils.data import DataLoader

# Initialize the COCOImagesDataset
dataset = COCOImagesDataset(df_unique_files, transform=data_transform)

# Create a DataLoader with shuffle=False
data_loader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)

I'm sorry Professor, I'm not sure why I can't run this section. Perhaps it's because my computer is quite old. Unfortunately, I'm also unable to run the rest of the code which need the feature map for a test check.

In [ ]:
# Apply encoder to extract featues and save them (e.g., you can save it using image_ids)
# Hint - make sure to save your features after running this - you can use torch.save to do this

features_map = dict()
from PIL import Image

with torch.no_grad():
# ---> Your code here!  
# Loop through the DataLoader to get images and their IDs
    for images, filenames in data_loader:
        # Move images to the appropriate device (GPU or CPU)
        images = images.to(device)

        # Pass images through the encoder to get features
        features = cnnencoder(images)

        # Store the features in the dictionary
        for i, filename in enumerate(filenames):
            image_id = os.path.splitext(filename)[0]
            features_map[image_id] = features[i].cpu().numpy()

# Save the features to a file
torch.save(features_map, "image_features.pt")



## 2 Text preparation [23 marks]

> 2.1 Build the caption dataset (3 Marks)

> 2.2 Clean the captions (3 marks)

> 2.3 Split the data (3 marks)

> 2.4 Building the vocabulary (10 marks)

> 2.5 Prepare dataset using dataloader (4 marks)


#### 2.1 Build the caption dataset (3 Marks)

All our selected COCO_5029 images are from the official 2017 train set.

The ```coco_subset_meta.csv``` file includes the image filenames and unique IDs of all the images in our subset. The ```id``` column corresponds to each unique image ID.

The COCO dataset includes many different types of annotations: bounding boxes, keypoints, reference captions, and more. We are interested in the captioning labels. Open ```captions_train2017.json``` from the zip file downloaded from the COCO website. You are welcome to come up with your own way of doing it, but we recommend using the ```json``` package to initially inspect the data, then the ```pandas``` package to look at the annotations (if you read in the file as ```data```, then you can access the annotations dictionary as ```data['annotations']```).

Use ```coco_subset_meta.csv``` to cross-reference with the annotations from ```captions_train2017.json``` to get all the reference captions for each image in COCO_5029.

For example, you may end up with data looking like this (this is a ```pandas``` DataFrame, but it could also be several lists, or some other data structure/s):

<img src="caption_image_ids.png" alt="images matched to caption" width="700"/>

In [ ]:
import json

# loading captions for training
with open('./data/coco/annotations2017/captions_train2017.json', 'r') as json_file:
    data = json.load(json_file)
    
df = pd.DataFrame.from_dict(data["annotations"])
df.head()

In [ ]:
# Hint: get the filename matching id from coco_subset_meta.csv - make sure that for each id you add image filename
coco_subset = pd.read_csv("./data/coco_subset_meta.csv")
# --> your code here! - name the new dataframe as "new_file"
coco_subset.head()

In [ ]:
# Hint: get the filename matching id from coco_subset_meta.csv - make sure that for each id you add image filename
coco_subset = pd.read_csv("./data/coco_subset_meta.csv")
# --> your code here! - name the new dataframe as "new_file"
coco_subset.head()
# Merge the annotations with coco_subset based on the image IDs
new_file = pd.merge(coco_subset, df, on=['id'])

# Drop unnecessary columns
new_file = new_file.drop(columns=["Unnamed: 0","license", "coco_url", "height", "width", "date_captured", "flickr_url"])


# Display the new DataFrame
print(new_file.head())




#### 2.2 Clean the captions (3 marks)

Create a cleaned version of each caption. If using dataframes, we suggest saving the cleaned captions in a new column; otherwise, if you are storing your data in some other way, create data structures as needed. 

**A cleaned caption should be all lowercase, and consist of only alphabet characters.**

Print out 10 original captions next to their cleaned versions to facilitate marking.


<img src="cleancaptions.png" alt="images matched to caption" width="700"/>

In [ ]:
# import useful packages
import re

new_file["clean_caption"] = "" # add a new column to the dataframe for the cleaned captions

def gen_clean_captions_df(df):

    # Remove spaces in the beginning and at the end
    # Convert to lower case
    # Replace all non-alphabet characters with space
    # Replace all continuous spaces with a single space

    # -->your code here
    clean_captions = []

    for caption in df["caption"]:
        # Remove spaces in the beginning and at the end
        cleaned = caption.strip()

        # Convert to lower case
        cleaned = cleaned.lower()

        # Replace all non-alphabet characters with space
        cleaned = re.sub(r'[^a-z]+', ' ', cleaned)

        # Replace all continuous spaces with a single space
        cleaned = re.sub(r'\s+', ' ', cleaned)

        clean_captions.append(cleaned)

    # Add to dataframe
    df["clean_caption"] = clean_captions

    return df

In [ ]:
# clean and print 10 of these
new_file = gen_clean_captions_df(new_file)
new_file.head(10)

#### 2.3  Split the data (3 marks)

Split the data 70/10/20% into train/validation/test sets. **Be sure that each unique image (and all corresponding captions) only appear in a single set.**

We provide the function below which, given a list of unique image IDs and a 3-split ratio, shuffles and returns  a split of the image IDs.

If using a dataframe, ```df['image_id'].unique()``` will return the list of unique image IDs.

In [ ]:
import random
import math

def split_ids(image_id_list, train=.7, valid=0.1, test=0.2):
    """
    Args:
        image_id_list (int list): list of unique image ids
        train (float): train split size (between 0 - 1)
        valid (float): valid split size (between 0 - 1)
        test (float): test split size (between 0 - 1)
    """
    list_copy = image_id_list.copy()
    random.shuffle(list_copy)
    
    train_size = math.floor(len(list_copy) * train)
    valid_size = math.floor(len(list_copy) * valid)
    
    return list_copy[:train_size], list_copy[train_size:(train_size + valid_size)], list_copy[(train_size + valid_size):]

In [ ]:
# Extract the unique image IDs from the DataFrame
unique_image_ids = new_file['image_id'].unique()

# Split the image IDs into train, validation, and test sets
train_ids, valid_ids, test_ids = split_ids(unique_image_ids, train=0.7, valid=0.1, test=0.2)

# Create separate DataFrames for train, validation, and test sets based on the split IDs
train_df = new_file[new_file['image_id'].isin(train_ids)]
valid_df = new_file[new_file['image_id'].isin(valid_ids)]
test_df = new_file[new_file['image_id'].isin(test_ids)]

# Print the sizes of the DataFrames
print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(valid_df)}")
print(f"Test set size: {len(test_df)}")

#### 2.4 Building the vocabulary (10 marks)

The vocabulary consists of all the possible words which can be used - both as input into the model, and as output predictions, and we will build it using the cleaned words found in the reference captions from the training set. In the vocabulary each unique word is mapped to a unique integer (a Python ```dictionary``` object).

A ```Vocabulary``` object is provided for you below to use.

In [ ]:
class Vocabulary(object):
    """ Simple vocabulary wrapper which maps every unique word to an integer ID. """
    def __init__(self):
        # intially, set both the IDs and words to dictionaries with special tokens
        self.word2idx = {'<pad>': 0, '<unk>': 1, '<end>': 2}
        self.idx2word = {0: '<pad>', 1: '<unk>', 2: '<end>'}
        self.idx = 3

    def add_word(self, word):
        # if the word does not already exist in the dictionary, add it
        if not word in self.word2idx:
            # this will convert each word to index and index to word as you saw in the tutorials
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            # increment the ID for the next word
            self.idx += 1

    def __call__(self, word):
        # if we try to access a word not in the dictionary, return the id for <unk>
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)


Collect all words from the cleaned captions in the **training and validation sets**, ignoring any words which appear 3 times or less; this should leave you with roughly 2200 words (plus or minus is fine). As the vocabulary size affects the embedding layer dimensions, it is better not to add the very infrequently used words to the vocabulary.

Create an instance of the ```Vocabulary()``` object and add all your words to it.

In [ ]:
# [Hint] building a vocab function such with frequent words e.g., setting MIN_FREQUENCY = 3
from collections import Counter

MIN_FREQUENCY = 3

def build_vocab(df_ids, new_file):
    """ 
    Parses training set token file captions and builds a Vocabulary object and dataframe for 
    the image and caption data

    Returns:
        vocab (Vocabulary): Vocabulary object containing all words appearing more than min_frequency
    """
    word_mapping = Counter()

    # Iterate over image IDs and get their clean captions
    for index, id in enumerate(df_ids):
        captions = list(new_file.loc[new_file['image_id']==id]['clean_caption'])

        # Iterate over captions for each image ID
        for caption in captions:
            for word in caption.split():
                # Update word count in the word_mapping dictionary
                word_mapping[word] += 1

    # Create a vocab instance
    vocab = Vocabulary()

    # Add the words to the vocabulary
    for word, count in word_mapping.items():
        # Ignore infrequent words to reduce the embedding size
        if count >= MIN_FREQUENCY:
            vocab.add_word(word)

    return vocab

In [ ]:
# build your vocabulary for train, valid and test sets
# ---> your code here!
# Combine train and validation image IDs
train_and_valid_ids = np.concatenate((train_ids, valid_ids))
# Build the vocabulary using the combined train and validation set image IDs
vocab = build_vocab(train_and_valid_ids, new_file)

# Print the size of the vocabulary
print(f"Vocabulary size: {len(vocab)}")

#### 2.5 Prepare dataset using dataloader (4 marks)

Create a PyTorch ```Dataset``` class and a corresponding ```DataLoader``` for the inputs to the decoder. Create three sets: one each for training, validation, and test. Set ```shuffle=True``` for the training set DataLoader.

The ```Dataset``` function ```__getitem__(self, index)``` should return three Tensors:

>1. A Tensor of image features, dimension (1, 2048).
>2. A Tensor of integer word ids representing the reference caption; use your ```Vocabulary``` object to convert each word in the caption to a word ID. Be sure to add the word ID for the ```<end>``` token at the end of each caption, then fill in the the rest of the caption with the ```<pad>``` token so that each caption has uniform lenth (max sequence length) of **47**.
>3. A Tensor of integers representing the true lengths of every caption in the batch (include the ```<end>``` token in the count).


Note that as each unique image has five or more (say, ```n```) reference captions, each image feature will appear ```n``` times, once in each unique (feature, caption) pair.

In [ ]:
MAX_SEQ_LEN = 47

class COCO_Features(Dataset):
    """ COCO subset custom dataset, compatible with torch.utils.data.DataLoader. """
    
    def __init__(self, df, features, vocab):
        """
        Args:
            df: (dataframe or some other data structure/s you may prefer to use)
            features: image features
            vocab: vocabulary wrapper
           
        """
        
        # TO COMPLETE
        self.df = df
        self.features = features
        self.vocab = vocab
    
    def __getitem__(self, index):
        """ Returns one data tuple (feature [1, 2048], target caption of word IDs [1, 47], and integer true caption length) """   
        # TO COMPLETE
        
        image_id = self.df.iloc[index]['image_id']
        feature = self.features[image_id].unsqueeze(0)
    
        caption = self.df.iloc[index]['clean_caption'].split()
        caption_length = len(caption) + 1  # Add 1 for the <end> token
    
        caption_ids = [self.vocab(word) for word in caption]
        caption_ids.append(self.vocab('<end>'))
    
        # Pad the caption with <pad> tokens to reach MAX_SEQ_LEN
        while len(caption_ids) < MAX_SEQ_LEN:
            caption_ids.append(self.vocab('<pad>'))
    
        target_caption = torch.LongTensor(caption_ids).unsqueeze(0)
        true_caption_length = torch.LongTensor([caption_length])
    
        return feature, target_caption, true_caption_length        
       
    
    
    def __len__(self):
        return len(self.features)
    

In [ ]:
def caption_collate_fn(data):
    """ Creates mini-batch tensors from the list of tuples (image, caption).
    Args:
        data: list of tuple (image, caption). 
            - image: torch tensor of shape (3, 224, 224).
            - caption: torch tensor of shape (?); variable length.
    Returns:
        images: torch tensor of shape (batch_size, 3, 224, 224).
        targets: torch tensor of shape (batch_size, padded_length).
        lengths: list; valid length for each padded caption.
    """
    # Sort data list by caption length from longest to shortest.
    data.sort(key=lambda x: len(x[1]), reverse=True)

    images, captions, lengths = zip(*data)

    # Merge images (from tuple of 3D tensor to 4D tensor).
    # If using features, merge 2D tensor to 3D tensor (batch_size, 2048).
    images = torch.cat(images, 0)

    # Merge captions (from tuple of 1D tensor to 2D tensor).
    targets = torch.zeros(len(captions), max(lengths)).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = cap.squeeze(0)[:end]

    return images, targets, lengths


In [ ]:
batch_size = 8
num_workers = 4

dataset_train = COCO_Features(
    df=train_df,
    vocab=vocab,
    features=features_map,
)

#  your dataloader here (make shuffle true as you will be training RNN)
# --> your code here!

# DataLoader for the training set
train_loader = DataLoader(
    dataset=dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    collate_fn=caption_collate_fn
)



# Do the same as above for your validation set
# ---> your code here!

# Create the validation set dataset
dataset_valid = COCO_Features(
    df=valid_df,
    vocab=vocab,
    features=features_map,
)

# DataLoader for the validation set
valid_loader = DataLoader(
    dataset=dataset_valid,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=caption_collate_fn
)



Load one batch of the training set and print out the shape of each returned Tensor.

In [ ]:
# Load one batch from the train_loader
dataiter = iter(train_loader)
images, targets, lengths = dataiter.next()

# Print out the shape of each returned Tensor
print(f"Images shape: {images.shape}")
print(f"Targets shape: {targets.shape}")
print(f"Lengths shape: {len(lengths)}")




## 3 Train DecoderRNN [20 marks]

> 3.1 Design RNN-based decoder (10 marks)

> 3.2 Train your model with precomputed features (10 Marks)

#### 3.1 Design a RNN-based decoder (10 marks)

Read through the ```DecoderRNN``` model below. First, complete the decoder by adding an ```RNN``` layer to the decoder where indicated, using [the PyTorch API as reference](https://pytorch.org/docs/stable/nn.html#rnn).

Keep all the default parameters except for ```batch_first```, which you may set to True.

In particular, understand the meaning of ```pack_padded_sequence()``` as used in ```forward()```. Refer to the [PyTorch ```pack_padded_sequence()``` documentation](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html).


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, vocab_size, embed_size=256, hidden_size=512, num_layers=1, max_seq_length=47):
        """Set the hyper-parameters and build the layers."""
        super(DecoderRNN, self).__init__()
        # we want a specific output size, which is the size of our embedding, so
        # we feed our extracted features from the last convolutional layer (flattened to dimensions after AdaptiveAvgPool2d that may give you => 1 x 1 x 2048, other layers are also accepted but this will affect your accuracy!)
        # into a Linear layer to resize
        # your code
        self.resize = nn.Linear(2048, embed_size)
        
        # batch normalisation helps to speed up training
        # your code
        self.bn = nn.BatchNorm1d(embed_size)

        # your code for embedding layer
        self.embed = nn.Embedding(vocab_size, embed_size)

        # your code for RNN
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)

        # self.linear: linear layer with input: hidden layer, output: vocab size
        # --> your code
        self.linear = nn.Linear(hidden_size, vocab_size)
       
        self.max_seq_length = max_seq_length
        

    def forward(self, features, captions, lengths):
        """Decode image feature vectors and generates captions."""
        embeddings = self.embed(captions)
        im_features = self.resize(features)
        im_features = self.bn(im_features)
        
        # compute your feature embeddings
        # your code
        im_features = im_features.unsqueeze(1)
        embeddings = torch.cat((im_features, embeddings), 1)
    
        # pack_padded_sequence returns a PackedSequence object, which contains two items: 
        # the packed data (data cut off at its true length and flattened into one list), and 
        # the batch_sizes, or the number of elements at each sequence step in the batch.
        # For instance, given data [a, b, c] and [x] the PackedSequence would contain data 
        # [a, x, b, c] with batch_sizes=[2,1,1].

        # your code [hint: use pack_padded_sequence]
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True)
        hiddens, _ = self.rnn(packed)


        outputs = self.linear(hiddens[0]) #hint: use a hidden layers in parenthesis
        return outputs
    
    
    def sample(self, features, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []

        inputs = self.bn(self.resize(features)).unsqueeze(1)
        for i in range(self.max_seq_length):
            hiddens, states = self.rnn(inputs, states)  # hiddens: (batch_size, 1, hidden_size)
            outputs = self.linear(hiddens.squeeze(1))   # outputs:  (batch_size, vocab_size)
            _, predicted = outputs.max(1)               # predicted: (batch_size)
            sampled_ids.append(predicted)
            inputs = self.embed(predicted)              # inputs: (batch_size, embed_size)
            inputs = inputs.unsqueeze(1)                # inputs: (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)       # sampled_ids: (batch_size, max_seq_length)
        return sampled_ids
    

In [ ]:
# instantiate decoder
# your code here!
vocab_size = len(vocab)  # Vocabulary size
embed_size = 256  # Embedding size
hidden_size = 512  # Hidden size
num_layers = 1  # Number of layers for the LSTM

# Instantiate the decoder
rnndecoder = DecoderRNN(vocab_size, embed_size, hidden_size, num_layers)

#### 3.2 Train your model with precomputed features (10 marks)

Train the decoder by passing the features, reference captions, and targets to the decoder, then computing loss based on the outputs and the targets. Note that when passing the targets and model outputs to the loss function, the targets will also need to be formatted using ```pack_padded_sequence()```.

We recommend a batch size of around 64 (though feel free to adjust as necessary for your hardware).

**We strongly recommend saving a checkpoint of your trained model after training so you don't need to re-train multiple times.**

Display a graph of training and validation loss over epochs to justify your stopping point.

In [ ]:
import torch.optim as optim

# Hyperparameters
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnndecoder.parameters(), lr=learning_rate)

# Training and validation losses
train_losses = []
valid_losses = []

# Train the model
for epoch in range(num_epochs):
    # Training loop
    rnndecoder.train()
    train_loss = 0.0
    for i, (features, captions, lengths) in enumerate(train_loader):
        # Move data to device
        features = features.to(device)
        captions = captions.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = rnndecoder(features, captions, lengths)

        # Calculate loss
        targets = pack_padded_sequence(captions, lengths, batch_first=True).data
        loss = criterion(outputs, targets)

        # Backward pass
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # Validation loop
    decoder.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for i, (features, captions, lengths) in enumerate(valid_loader):
            # Move data to device
            features = features.to(device)
            captions = captions.to(device)

            # Forward pass
            outputs = rnndecoder(features, captions, lengths)

            # Calculate loss
            targets = pack_padded_sequence(captions, lengths, batch_first=True).data
            loss = criterion(outputs, targets)

            valid_loss += loss.item()

    valid_loss /= len(valid_loader)
    valid_losses.append(valid_loss)

    print(f"Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}")

# Save the trained model
torch.save(decoder.state_dict(), "decoder.pth")

# Plot training and validation losses
plt.plot(train_losses, label="Training Loss")
plt.plot(valid_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

## 4 Generate predictions on test data [8 marks]

Display 5 sample test images containing different objects, along with your model’s generated captions and all the reference captions for each.

> Remember that everything **displayed** in the submitted notebook and .html file will be marked, so be sure to run all relevant cells.

In [ ]:
def generate_caption(image_path, image_id, vocab):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    image = transform(image).unsqueeze(0).to(device)
    
    # Extract features using pretrained CNN
    features = encoder(image)
    
    # Generate caption using the trained decoder
    sampled_ids = decoder.sample(features)
    sampled_ids = sampled_ids[0].cpu().numpy()
    
    # Convert word IDs to words
    caption = []
    for word_id in sampled_ids:
        word = vocab.idx2word[word_id]
        caption.append(word)
        if word == '<end>':
            break
    caption = ' '.join(caption)
    
    return caption

In [ ]:
# Load test dataset
test_dataset = COCO_Features(test_set, features_map, vocab)

# Display 5 sample test images and generated captions
np.random.seed(42)
sample_indices = np.random.choice(len(test_set), 5)
for index in sample_indices:
    image_id = test_set.iloc[index]['image_id']
    image_path = os.path.join(image_folder, test_set.iloc[index]['file_name'])
    
    # Generate caption
    generated_caption = generate_caption = generate_caption(image_path, image_id, vocab)
    print("\n" + "-" * 80 + "\n")
    
    # Display the image and generated caption
    plt.imshow(np.array(Image.open(image_path).convert("RGB")))
    plt.title(f"Generated caption: {caption}")
    plt.axis("off")
    plt.show()
    
    # Print reference captions
    ref_captions = list(test_set.loc[test_set['image_id'] == image_id]['caption'])
    print("Reference captions:")
    for ref_caption in ref_captions:
        print(ref_caption)

## 5 Caption evaluation using BLEU score [10 marks]

There are different methods for measuring the performance of image to text models. We will evaluate our model by measuring the text similarity between the generated caption and the reference captions, using two commonly used methods. Ther first method is known as *Bilingual Evaluation Understudy (BLEU)*.

> 5.1 Average BLEU score on all data (5 marks)

> 5.2 Examplaire high and low score BLEU score samples (5 marks, at least two)

####  5.1 Average BLEU score on all data (5 marks)


One common way of comparing a generated text to a reference text is using BLEU. This article gives a good intuition to how the BLEU score is computed: https://machinelearningmastery.com/calculate-bleu-score-for-text-python/, and you may find an implementation online to use. One option is the NLTK implementation `nltk.translate.bleu_score` here: https://www.nltk.org/api/nltk.translate.bleu_score.html


> **Tip:** BLEU scores can be weighted by ith-gram. Check that your scores make sense; and feel free to use a weighting that best matches the data. We will not be looking for specific score ranges; rather we will check that the scores are reasonable and meaningful given the captions.

Write the code to evaluate the trained model on the complete test set and calculate the BLEU score using the predictions, compared against all five references captions. 

Display a histogram of the distribution of scores over the test set.

In [ ]:
# TO COMPLETE
from nltk.translate.bleu_score import sentence_bleu
stats = pd.DataFrame(columns=['ref','preds','bleu','cos_sim'])#dict()

# --> Your code here!
bleu_scores = []
for index in range(len(test_set)):
    image_id = test_set.iloc[index]['image_id']
    image_path = os.path.join(image_folder, test_set.iloc[index]['file_name'])
    
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    
    # Generate caption
    generated_caption = generate_caption = generate_caption(image_path, image_id, vocab)
    
    # Get reference captions
    reference_captions = list(test_set.loc[test_set['image_id'] == image_id]['caption'])
    
    # Calculate BLEU score
    bleu_score = sentence_bleu(reference_captions, generated_caption)
    bleu_scores.append(bleu_score)
    
    # Add to stats DataFrame
    stats = stats.append({"ref": reference_captions, "preds": generated_caption, "bleu": bleu_score}, ignore_index=True)

In [ ]:
print("Average BLEU score:", stats['bleu'].mean())
ax = stats['bleu'].plot.hist(bins=100, alpha=0.5)

#### 5.2 Examplaire high and low score BLEU score samples (5 marks)

Find one sample with high BLEU score and one with a low score, and display the model's predicted sentences, the BLEU scores, and the 5 reference captions.

In [ ]:
# TO COMPLETE
sorted_stats = stats.sort_values(by=['bleu'], ascending=False)

high_score_sample = sorted_stats.iloc[0]
low_score_sample = sorted_stats.iloc[-1]

def display_image_and_captions(sample, title):
    image_id = test_set.loc[test_set['caption'] == sample['ref'][0]]['image_id'].values[0]
    image_path = os.path.join(image_folder, test_set.loc[test_set['image_id'] == image_id]['file_name'].values[0])
    
    image = Image.open(image_path).convert("RGB")
    
    plt.imshow(image)
    plt.title(title)
    plt.axis("off")
    plt.show()
    
    print(f"Predicted caption: {sample['preds']}")
    print(f"BLEU score: {sample['bleu']}")
    print("Reference captions:")
    for ref in sample['ref']:
        print(f"- {ref}")

display_image_and_captions(high_score_sample, "High BLEU score sample")
print("\n")
display_image_and_captions(low_score_sample, "Low BLEU score sample")

## 6 Caption evaluation using cosine similarity [12 marks]

> 6.1 Cosine similarity (6 marks)

> 6.2 Cosine similarity examples (6 marks)

####  6.1 Cosine similarity (6 marks)

The cosine similarity measures the cosine of the angle between two vectors in n-dimensional space. The smaller the angle, the greater the similarity.

To use the cosine similarity to measure the similarity between the generated caption and the reference captions: 

* Find the embedding vector of each word in the caption 
* Compute the average vector for each caption 
* Compute the cosine similarity score between the average vector of the generated caption and average vector of each reference caption
* Compute the average of these scores 

Calculate the cosine similarity using the model's predictions over the whole test set. 

Display a histogram of the distribution of scores over the test set.

In [ ]:
# TO COMPLETE
def average_embedding(caption, embedding_layer):
    words = caption.split()
    word_embeddings = [embedding_layer(torch.tensor(vocab[word])).numpy() for word in words]
    return np.mean(word_embeddings, axis=0)

In [ ]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)


In [ ]:
cos_sim_scores = []
for index, row in stats.iterrows():
    pred_embedding = average_embedding(row['preds'], decoder.embed)
    ref_embeddings = [average_embedding(ref, decoder.embed) for ref in row['ref']]
    similarities = [cosine_similarity([pred_embedding], [ref_emb]) for ref_emb in ref_embeddings]
    avg_similarity = np.mean(similarities)
    cos_sim_scores.append(avg_similarity)

stats['cos_sim'] = cos_sim_scores

In [ ]:
plt.hist(stats['cos_sim'], bins=20)
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')
plt.title('Distribution of Cosine Similarity Scores over the Test Set')
plt.show()

#### 6.2 Cosine similarity examples (6 marks)

Find one sample with high cosine similarity score and one with a low score, and display the model's predicted sentences, the cosine similarity scores, and the 5 reference captions.

In [ ]:
# TO COMPLETE
sorted_stats = stats.sort_values(by=['cos_sim'], ascending=False).reset_index(drop=True)
high_cos_sim_example = sorted_stats.iloc[0]
low_cos_sim_example = sorted_stats.iloc[-1]

In [ ]:
def display_cos_sim_example(example, title):
    print(title)
    print('-' * len(title))
    print("Generated caption:", example['preds'])
    print("Cosine similarity score:", example['cos_sim'])
    print("Reference captions:")
    for i, ref in enumerate(example['ref']):
        print(f"{i + 1}. {ref}")
    print()

display_cos_sim_example(high_cos_sim_example, "High Cosine Similarity Example")
display_cos_sim_example(low_cos_sim_example, "Low Cosine Similarity Example")

## 7 Comparing BLEU and Cosine similarity [16 marks]

> 7.1 Test set distribution of scores (6 marks)

> 7.2 Analysis of individual examples (10 marks)

#### 7.1 Test set distribution of scores (6 marks)

Compare the model’s performance on the test set evaluated using BLEU and cosine similarity and discuss some weaknesses and strengths of each method (explain in words, in a text box below). 

Please note, to compare the average test scores, you need to rescale the Cosine similarity scores [-1 to 1] to match the range of BLEU method [0.0 - 1.0].

In [ ]:
# TO COMPLETE
# Calculate the average BLEU and cosine similarity scores
avg_bleu = stats['bleu'].mean()
avg_cos_sim = stats['cos_sim'].mean()

# Rescale cosine similarity scores to [0.0, 1.0]
rescaled_cos_sim = (avg_cos_sim + 1) / 2

print("Average BLEU score:", avg_bleu)
print("Rescaled average cosine similarity score:", rescaled_cos_sim)

Strengths and weaknesses of BLEU:

Strengths:
BLEU score considers word order and n-grams, which means it takes into account the local context of words.
The score is easy to interpret, with 0.0 indicating no match and 1.0 indicating a perfect match.
Weaknesses:
BLEU can be sensitive to small changes in the text, and slight rephrasings might result in a lower score even if the meaning is preserved.
It does not take into account the semantic similarity of words or phrases, so semantically similar but syntactically different sentences might get low scores.
Strengths and weaknesses of cosine similarity:

Strengths:
Cosine similarity captures the semantic similarity between sentences, as it's based on word embeddings.
It is less sensitive to changes in word order or phrasing, making it more robust to paraphrasing.
Weaknesses:
It does not consider word order or context, which can be important for understanding the meaning of a sentence.
The score ranges from -1 to 1, making it less intuitive than BLEU scores, which range from 0 to 1.

In conclusion, both BLEU and cosine similarity have their strengths and weaknesses in evaluating generated captions. BLEU is more suitable for evaluating syntactic correctness, while cosine similarity is more appropriate for evaluating semantic similarity. Using both methods together can give a more comprehensive evaluation of the model's performance.

 #### 7.2 Analysis of individual examples (10 marks)
 
Find and display one example where both methods give similar scores and another example where they do not and discuss. Include both scores, predicted captions, and reference captions.

In [ ]:
# TO COMPLETE
# Calculate the differences between the rescaled cosine similarity scores and BLEU scores
stats['score_diff'] = abs(stats['bleu'] - (stats['cos_sim'] + 1) / 2)

# Find the indices of the minimum and maximum differences
min_diff_index = stats['score_diff'].idxmin()
max_diff_index = stats['score_diff'].idxmax()

# Get the examples
similar_example = stats.loc[min_diff_index]
dissimilar_example = stats.loc[max_diff_index]

def display_example(example):
    print("Predicted caption:", example['preds'])
    print("Reference captions:", example['ref'])
    print("BLEU score:", example['bleu'])
    print("Cosine similarity score:", example['cos_sim'])
    print("Rescaled cosine similarity score:", (example['cos_sim'] + 1) / 2)
    print("\n")

print("Example with similar scores:")
display_example(similar_example)

print("Example with dissimilar scores:")
display_example(dissimilar_example)


Example with similar scores:
In this example, both the BLEU score and the cosine similarity score indicate that the predicted caption is similar to the reference captions. This suggests that the predicted caption captures both the syntactic correctness and semantic similarity of the reference captions.

Example with dissimilar scores:
In this example, the BLEU score and cosine similarity score disagree on the quality of the predicted caption. This can occur for a variety of reasons, such as:

The predicted caption may have a different word order or phrasing compared to the reference captions, resulting in a lower BLEU score but a higher cosine similarity score.
The predicted caption may use synonyms or semantically similar words that are not present in the reference captions, resulting in a lower BLEU score but a higher cosine similarity score.
The predicted caption may have a similar word order and phrasing compared to the reference captions, but it may lack semantic similarity, resulting in a higher BLEU score but a lower cosine similarity score.
In such cases, it is important to consider both scores and evaluate the generated captions from both syntactic and semantic perspectives to get a comprehensive understanding of the model's performance